# Import statements

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import yfinance as yf
import datetime

# Read in NYSE tickers, get 1000 largest companies, remove those with IPO after 12/31/2009 and those that yfinance has no data for, keep data starting from day at which all companies are publicly traded, and save data 

In [22]:
df = pd.read_csv('nyse_tickers.csv')
df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$126.97,-1.0900,-0.851%,3.754526e+10,United States,1999.0,733982,Industrials,Electrical Products
1,AA,Alcoa Corporation Common Stock,$35.28,-0.8600,-2.38%,6.292347e+09,NaN,2016.0,2083531,Industrials,Aluminum
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,$10.43,0.0000,0.00%,0.000000e+00,NaN,2021.0,9816,Industrials,Metal Fabrications
3,AAIC,Arlington Asset Investment Corp Class A (new),$2.59,-0.0200,-0.766%,7.345356e+07,United States,NaN,3472,Real Estate,Real Estate Investment Trusts
4,AAIC^B,Arlington Asset Investment Corp 7.00%,$17.48,-0.0021,-0.012%,NaN,United States,NaN,100,NaN,NaN


In [23]:
df.dtypes

Symbol         object
Name           object
Last Sale      object
Net Change    float64
% Change       object
Market Cap    float64
Country        object
IPO Year      float64
Volume          int64
Sector         object
Industry       object
dtype: object

In [24]:
df.sort_values(by=['Market Cap'],ascending=False,inplace=True)
df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
450,BRK/A,Berkshire Hathaway Inc.,$490688.50,-1311.500,-0.267%,7.217410e+11,United States,NaN,4678,NaN,NaN
451,BRK/B,Berkshire Hathaway Inc.,$321.56,-1.430,-0.443%,7.094615e+11,United States,NaN,1132868,NaN,NaN
1474,JNJ,Johnson & Johnson Common Stock,$159.93,-1.720,-1.064%,4.989564e+11,United States,NaN,2526818,Health Care,Biotechnology: Pharmaceutical Preparations
2755,V,Visa Inc.,$230.67,-0.600,-0.259%,4.848190e+11,United States,NaN,1732859,Consumer Discretionary,Business Services
2721,UNH,UnitedHealth Group Incorporated Common Stock (DE),$486.815,-3.255,-0.664%,4.535181e+11,United States,NaN,1228681,Health Care,Medical Specialities


In [27]:
symbols = df.iloc[:1000,:]['Symbol'] n#1000 symbols corresponding to largest companies on nyse

In [30]:
new = []
symbols = list(symbols)
for s in symbols:
    new.append(s.replace('^','-').replace('/','-').replace('.','-'))
symbols = new[:]

In [43]:
yf.Ticker('brk-b').history(period='max').head(1).index[0]

Timestamp('1996-05-09 00:00:00-0400', tz='America/New_York')

In [45]:
remove = []
for s in symbols:
    hist_df = yf.Ticker(s.upper()).history(period='max')
    if hist_df.shape[0] == 0:
        remove.append(s)
        continue
    year = hist_df.head(1).index[0].year
    if year >= 2010:
        remove.append(s)

In [46]:
symbols = set(symbols).difference(set(remove))

In [47]:
symbols

{'A',
 'AA',
 'AAP',
 'AB',
 'ABB',
 'ABC',
 'ABEV',
 'ABG',
 'ABT',
 'ACM',
 'ACN',
 'ADC',
 'ADM',
 'AEE',
 'AEG',
 'AEL',
 'AEM',
 'AER',
 'AES',
 'AFG',
 'AFL',
 'AGCO',
 'AGI',
 'AGO',
 'AIG',
 'AIT',
 'AIZ',
 'AJG',
 'AJRD',
 'ALB',
 'ALE',
 'ALK',
 'ALL',
 'ALV',
 'AME',
 'AMG',
 'AMN',
 'AMP',
 'AMT',
 'AMX',
 'AN',
 'AON',
 'AOS',
 'APD',
 'APH',
 'AQN',
 'ARE',
 'ARW',
 'ASGN',
 'ASH',
 'ASR',
 'ASX',
 'ATI',
 'ATO',
 'ATR',
 'AU',
 'AVA',
 'AVB',
 'AVNT',
 'AVY',
 'AWK',
 'AWR',
 'AXP',
 'AXS',
 'AYI',
 'AZO',
 'BA',
 'BAC',
 'BAK',
 'BALL',
 'BAP',
 'BAX',
 'BBD',
 'BBVA',
 'BBWI',
 'BBY',
 'BC',
 'BCE',
 'BCH',
 'BCO',
 'BCS',
 'BDC',
 'BDX',
 'BEN',
 'BEP',
 'BG',
 'BHP',
 'BIO',
 'BIP',
 'BK',
 'BKH',
 'BLDR',
 'BLK',
 'BMI',
 'BMO',
 'BMY',
 'BN',
 'BNS',
 'BP',
 'BR',
 'BRK-A',
 'BRK-B',
 'BRO',
 'BSAC',
 'BSBR',
 'BSX',
 'BTI',
 'BUD',
 'BWA',
 'BX',
 'BXP',
 'BYD',
 'C',
 'CACI',
 'CADE',
 'CAE',
 'CAG',
 'CAH',
 'CAT',
 'CB',
 'CBRE',
 'CBT',
 'CCI',
 'CCJ',
 'CCK',

In [49]:
remove

['ABBV',
 'BABA',
 'NOW',
 'ZTS',
 'SHOP',
 'UBER',
 'HCA',
 'SNOW',
 'RACE',
 'STLA',
 'TAK',
 'SE',
 'MPC',
 'GM',
 'PSX',
 'ANET',
 'KKR',
 'HLN',
 'CTVA',
 'ALC',
 'HLT',
 'KMI',
 'DOW',
 'APO',
 'CARR',
 'IQV',
 'OTIS',
 'SQ',
 'MPLX',
 'DELL',
 'VICI',
 'NTR',
 'FERG',
 'LYB',
 'CPNG',
 'SPOT',
 'BBDO',
 'NU',
 'VEEV',
 'DASH',
 'YUMC',
 'KEYS',
 'APTV',
 'RBLX',
 'ZTO',
 'IR',
 'HUBS',
 'QSR',
 'FTV',
 'FTS',
 'BEKE',
 'INVH',
 'PLTR',
 'XYL',
 'CNHI',
 'HWM',
 'HPE',
 'PBA',
 'FLT',
 'LW',
 'NET',
 'PAYC',
 'TRGP',
 'AGR',
 'GFL',
 'AMCR',
 'CHWY',
 'PINS',
 'ARES',
 'NIO',
 'SNAP',
 'AVTR',
 'DT',
 'EPAM',
 'BAM',
 'TRU',
 'AMH',
 'TME',
 'PEN',
 'BAH',
 'U',
 'CFG',
 'ACI',
 'SYF',
 'REXR',
 'BURL',
 'TOST',
 'GDDY',
 'CRBG',
 'UI',
 'ONON',
 'AGL',
 'BILL',
 'IOT',
 'FND',
 'WES',
 'ARMK',
 'BJ',
 'COTY',
 'ALLE',
 'PFGC',
 'VST',
 'XPEV',
 'USFD',
 'VIPS',
 'TWLO',
 'CDAY',
 'BKI',
 'INSP',
 'FBIN',
 'EQH',
 'COLD',
 'PATH',
 'PCOR',
 'HII',
 'AMC',
 'STVN',
 'KNSL',
 'ALLY

In [55]:
help(yf)

Help on package yfinance:

NAME
    yfinance

DESCRIPTION
    # -*- coding: utf-8 -*-
    #
    # yfinance - market data downloader
    # https://github.com/ranaroussi/yfinance
    #
    # Copyright 2017-2019 Ran Aroussi
    #
    # Licensed under the Apache License, Version 2.0 (the "License");
    # you may not use this file except in compliance with the License.
    # You may obtain a copy of the License at
    #
    #     http://www.apache.org/licenses/LICENSE-2.0
    #
    # Unless required by applicable law or agreed to in writing, software
    # distributed under the License is distributed on an "AS IS" BASIS,
    # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    # See the License for the specific language governing permissions and
    # limitations under the License.
    #

PACKAGE CONTENTS
    base
    data
    exceptions
    multi
    scrapers (package)
    shared
    ticker
    tickers
    utils
    version

CLASSES
    builtins.object
        yf

In [94]:
sym_str = ''
for s in symbols:
    sym_str+=s+' '
data = yf.download(sym_str,period='max')

[*********************100%***********************]  719 of 719 completed


In [57]:
data.head()

Adj Close                                             ... Volume  \
                   A        AA AAP  AB ABB ABC ABEV ABG ABT ACM  ...    WTS   
Date                                                             ...          
1962-01-02       NaN  1.527621 NaN NaN NaN NaN  NaN NaN NaN NaN  ...    NaN   
1962-01-03       NaN  1.550988 NaN NaN NaN NaN  NaN NaN NaN NaN  ...    NaN   
1962-01-04       NaN  1.550988 NaN NaN NaN NaN  NaN NaN NaN NaN  ...    NaN   
1962-01-05       NaN  1.548068 NaN NaN NaN NaN  NaN NaN NaN NaN  ...    NaN   
1962-01-08       NaN  1.495492 NaN NaN NaN NaN  NaN NaN NaN NaN  ...    NaN   

                                                     
            WU WWE  WY   X      XOM XPO YPF YUM ZBH  
Date                                                 
1962-01-02 NaN NaN NaN NaN   902400 NaN NaN NaN NaN  
1962-01-03 NaN NaN NaN NaN  1200000 NaN NaN NaN NaN  
1962-01-04 NaN NaN NaN NaN  1088000 NaN NaN NaN NaN  
1962-01-05 NaN NaN NaN NaN  1222400 NaN NaN NaN NaN  
1962-01-08 NaN NaN NaN NaN  1388800 NaN NaN NaN NaN  

[5 rows x 4314 columns]

In [76]:
for i in range(data.shape[0]):
    day = data.iloc[i,:]
    if True not in list(day.isna()):
        pts = day.name
        print(pts)
        break

2009-12-29 00:00:00


In [95]:
data = data.loc[pts:,:]

In [96]:
data.head()

Adj Close                                                        \
                    A         AA        AAP        AB        ABB        ABC   
Date                                                                          
2009-12-29  19.640217  35.237846  37.049255  9.560406  12.084818  21.716007   
2009-12-30  20.138828  35.831367  37.276497  9.712652  12.072253  21.666595   
2009-12-31  20.119402  35.435684  36.794739  9.723032  11.996881  21.468946   
2010-01-04  20.268339  36.600769  36.703854  9.826835  12.336061  21.930115   
2010-01-05  20.048172  35.457672  36.485718  9.816459  12.417716  21.773653   

                                                   ...    Volume             \
                ABEV    ABG        ABT        ACM  ...       WTS         WU   
Date                                               ...                        
2009-12-29  2.183232  11.78  19.265392  27.797508  ...  144400.0  3134100.0   
2009-12-30  2.174173  11.81  19.364801  27.402514  ...  223300.0  2873600.0   
2009-12-31  2.172915  11.53  19.169525  27.155643  ...   55700.0  2418200.0   
2010-01-04  2.264763  11.83  19.336397  27.797508  ...  179100.0  4277900.0   
2010-01-05  2.249161  12.39  19.180170  28.103621  ...  164400.0  3914900.0   

                                                                         \
                 WWE         WY           X       XOM      XPO      YPF   
Date                                                                      
2009-12-29  111400.0   760100.0   8529900.0  19985700  69683.0   3600.0   
2009-12-30  148700.0   647100.0   8010200.0  19860600  18722.0   2600.0   
2009-12-31  106000.0   702000.0   3589500.0  18852300  36648.0  11300.0   
2010-01-04  159800.0  1832400.0  10339400.0  27809100  28336.0   5100.0   
2010-01-05  354600.0  1724500.0   9003200.0  30174700  44744.0   5300.0   

                                  
                  YUM        ZBH  
Date                              
2009-12-29  2844734.0   607906.0  
2009-12-30  1886196.0   594413.0  
2009-12-31  1790217.0   579169.0  
2010-01-04  2962274.0   805872.0  
2010-01-05  3298757.0  1769643.0  

[5 rows x 4314 columns]

In [97]:
months = []
days = []
years = []
index = data.index
for date in index:
    months.append(date.month)
    days.append(date.day)
    years.append(date.year)
months = pd.Series(months,index=index)
years = pd.Series(years,index=index)
days = pd.Series(days,index=index)
data['Month'] = months
data['Day'] = days
data['Year'] = years
data.head()

Adj Close                                                        \
                    A         AA        AAP        AB        ABB        ABC   
Date                                                                          
2009-12-29  19.640217  35.237846  37.049255  9.560406  12.084818  21.716007   
2009-12-30  20.138828  35.831367  37.276497  9.712652  12.072253  21.666595   
2009-12-31  20.119402  35.435684  36.794739  9.723032  11.996881  21.468946   
2010-01-04  20.268339  36.600769  36.703854  9.826835  12.336061  21.930115   
2010-01-05  20.048172  35.457672  36.485718  9.816459  12.417716  21.773653   

                                                   ...     Volume              \
                ABEV    ABG        ABT        ACM  ...         WY           X   
Date                                               ...                          
2009-12-29  2.183232  11.78  19.265392  27.797508  ...   760100.0   8529900.0   
2009-12-30  2.174173  11.81  19.364801  27.402514  ...   647100.0   8010200.0   
2009-12-31  2.172915  11.53  19.169525  27.155643  ...   702000.0   3589500.0   
2010-01-04  2.264763  11.83  19.336397  27.797508  ...  1832400.0  10339400.0   
2010-01-05  2.249161  12.39  19.180170  28.103621  ...  1724500.0   9003200.0   

                                                             Month Day  Year  
                 XOM      XPO      YPF        YUM        ZBH                  
Date                                                                          
2009-12-29  19985700  69683.0   3600.0  2844734.0   607906.0    12  29  2009  
2009-12-30  19860600  18722.0   2600.0  1886196.0   594413.0    12  30  2009  
2009-12-31  18852300  36648.0  11300.0  1790217.0   579169.0    12  31  2009  
2010-01-04  27809100  28336.0   5100.0  2962274.0   805872.0     1   4  2010  
2010-01-05  30174700  44744.0   5300.0  3298757.0  1769643.0     1   5  2010  

[5 rows x 4317 columns]

In [98]:
data['Date']=pd.Series(data.index,index=data.index)
data.head()

Adj Close                                                        \
                    A         AA        AAP        AB        ABB        ABC   
Date                                                                          
2009-12-29  19.640217  35.237846  37.049255  9.560406  12.084818  21.716007   
2009-12-30  20.138828  35.831367  37.276497  9.712652  12.072253  21.666595   
2009-12-31  20.119402  35.435684  36.794739  9.723032  11.996881  21.468946   
2010-01-04  20.268339  36.600769  36.703854  9.826835  12.336061  21.930115   
2010-01-05  20.048172  35.457672  36.485718  9.816459  12.417716  21.773653   

                                                   ...      Volume            \
                ABEV    ABG        ABT        ACM  ...           X       XOM   
Date                                               ...                         
2009-12-29  2.183232  11.78  19.265392  27.797508  ...   8529900.0  19985700   
2009-12-30  2.174173  11.81  19.364801  27.402514  ...   8010200.0  19860600   
2009-12-31  2.172915  11.53  19.169525  27.155643  ...   3589500.0  18852300   
2010-01-04  2.264763  11.83  19.336397  27.797508  ...  10339400.0  27809100   
2010-01-05  2.249161  12.39  19.180170  28.103621  ...   9003200.0  30174700   

                                                   Month Day  Year       Date  
                XPO      YPF        YUM        ZBH                             
Date                                                                           
2009-12-29  69683.0   3600.0  2844734.0   607906.0    12  29  2009 2009-12-29  
2009-12-30  18722.0   2600.0  1886196.0   594413.0    12  30  2009 2009-12-30  
2009-12-31  36648.0  11300.0  1790217.0   579169.0    12  31  2009 2009-12-31  
2010-01-04  28336.0   5100.0  2962274.0   805872.0     1   4  2010 2010-01-04  
2010-01-05  44744.0   5300.0  3298757.0  1769643.0     1   5  2010 2010-01-05  

[5 rows x 4318 columns]

In [99]:
data.reset_index(drop=True,inplace=True)
data.head()

Adj Close                                                                  \
           A         AA        AAP        AB        ABB        ABC      ABEV   
0  19.640217  35.237846  37.049255  9.560406  12.084818  21.716007  2.183232   
1  20.138828  35.831367  37.276497  9.712652  12.072253  21.666595  2.174173   
2  20.119402  35.435684  36.794739  9.723032  11.996881  21.468946  2.172915   
3  20.268339  36.600769  36.703854  9.826835  12.336061  21.930115  2.264763   
4  20.048172  35.457672  36.485718  9.816459  12.417716  21.773653  2.249161   

                                ...      Volume                              \
     ABG        ABT        ACM  ...           X       XOM      XPO      YPF   
0  11.78  19.265392  27.797508  ...   8529900.0  19985700  69683.0   3600.0   
1  11.81  19.364801  27.402514  ...   8010200.0  19860600  18722.0   2600.0   
2  11.53  19.169525  27.155643  ...   3589500.0  18852300  36648.0  11300.0   
3  11.83  19.336397  27.797508  ...  10339400.0  27809100  28336.0   5100.0   
4  12.39  19.180170  28.103621  ...   9003200.0  30174700  44744.0   5300.0   

                        Month Day  Year       Date  
         YUM        ZBH                             
0  2844734.0   607906.0    12  29  2009 2009-12-29  
1  1886196.0   594413.0    12  30  2009 2009-12-30  
2  1790217.0   579169.0    12  31  2009 2009-12-31  
3  2962274.0   805872.0     1   4  2010 2010-01-04  
4  3298757.0  1769643.0     1   5  2010 2010-01-05  

[5 rows x 4318 columns]

In [100]:
data.columns

MultiIndex([('Adj Close',    'A'),
            ('Adj Close',   'AA'),
            ('Adj Close',  'AAP'),
            ('Adj Close',   'AB'),
            ('Adj Close',  'ABB'),
            ('Adj Close',  'ABC'),
            ('Adj Close', 'ABEV'),
            ('Adj Close',  'ABG'),
            ('Adj Close',  'ABT'),
            ('Adj Close',  'ACM'),
            ...
            (   'Volume',    'X'),
            (   'Volume',  'XOM'),
            (   'Volume',  'XPO'),
            (   'Volume',  'YPF'),
            (   'Volume',  'YUM'),
            (   'Volume',  'ZBH'),
            (    'Month',     ''),
            (      'Day',     ''),
            (     'Year',     ''),
            (     'Date',     '')],
           length=4318)

In [101]:
new_cols = []
cur_cols = list(data.columns)
for tup in cur_cols:
    if tup[1] == '':
        new_cols.append(tup[0])
    else:
        new_cols.append(tup[0]+': ' + tup[1])
data.columns = new_cols
data.head()

,Adj Close: A,Adj Close: AA,Adj Close: AAP,Adj Close: AB,Adj Close: ABB,Adj Close: ABC,Adj Close: ABEV,Adj Close: ABG,Adj Close: ABT,Adj Close: ACM,...,Volume: X,Volume: XOM,Volume: XPO,Volume: YPF,Volume: YUM,Volume: ZBH,Month,Day,Year,Date
0,19.640217,35.237846,37.049255,9.560406,12.084818,21.716007,2.183232,11.78,19.265392,27.797508,...,8529900.0,19985700,69683.0,3600.0,2844734.0,607906.0,12,29,2009,2009-12-29
1,20.138828,35.831367,37.276497,9.712652,12.072253,21.666595,2.174173,11.81,19.364801,27.402514,...,8010200.0,19860600,18722.0,2600.0,1886196.0,594413.0,12,30,2009,2009-12-30
2,20.119402,35.435684,36.794739,9.723032,11.996881,21.468946,2.172915,11.53,19.169525,27.155643,...,3589500.0,18852300,36648.0,11300.0,1790217.0,579169.0,12,31,2009,2009-12-31
3,20.268339,36.600769,36.703854,9.826835,12.336061,21.930115,2.264763,11.83,19.336397,27.797508,...,10339400.0,27809100,28336.0,5100.0,2962274.0,805872.0,1,4,2010,2010-01-04
4,20.048172,35.457672,36.485718,9.816459,12.417716,21.773653,2.249161,12.39,19.180170,28.103621,...,9003200.0,30174700,44744.0,5300.0,3298757.0,1769643.0,1,5,2010,2010-01-05


In [102]:
data.to_csv('data.csv',index=False)

# End

# Read in assets' data

In [103]:
data = pd.read_csv('data.csv')
data.head()

,Adj Close: A,Adj Close: AA,Adj Close: AAP,Adj Close: AB,Adj Close: ABB,Adj Close: ABC,Adj Close: ABEV,Adj Close: ABG,Adj Close: ABT,Adj Close: ACM,...,Volume: X,Volume: XOM,Volume: XPO,Volume: YPF,Volume: YUM,Volume: ZBH,Month,Day,Year,Date
0,19.640217,35.237846,37.049255,9.560406,12.084818,21.716007,2.183232,11.78,19.265392,27.797508,...,8529900.0,19985700,69683.0,3600.0,2844734.0,607906.0,12,29,2009,2009-12-29
1,20.138828,35.831367,37.276497,9.712652,12.072253,21.666595,2.174173,11.81,19.364801,27.402514,...,8010200.0,19860600,18722.0,2600.0,1886196.0,594413.0,12,30,2009,2009-12-30
2,20.119402,35.435684,36.794739,9.723032,11.996881,21.468946,2.172915,11.53,19.169525,27.155643,...,3589500.0,18852300,36648.0,11300.0,1790217.0,579169.0,12,31,2009,2009-12-31
3,20.268339,36.600769,36.703854,9.826835,12.336061,21.930115,2.264763,11.83,19.336397,27.797508,...,10339400.0,27809100,28336.0,5100.0,2962274.0,805872.0,1,4,2010,2010-01-04
4,20.048172,35.457672,36.485718,9.816459,12.417716,21.773653,2.249161,12.39,19.180170,28.103621,...,9003200.0,30174700,44744.0,5300.0,3298757.0,1769643.0,1,5,2010,2010-01-05


# End

# Get market data and save it

In [109]:
market_df = yf.Ticker('SPY').history(period='max')
market_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
1993-01-29 00:00:00-05:00,25.140211,25.140211,25.015135,25.122343,1003200,0.0,0.0,0.0
1993-02-01 00:00:00-05:00,25.140212,25.301023,25.140212,25.301023,480500,0.0,0.0,0.0
1993-02-02 00:00:00-05:00,25.283158,25.372497,25.229554,25.354630,201300,0.0,0.0,0.0
1993-02-03 00:00:00-05:00,25.390369,25.640520,25.372501,25.622652,529400,0.0,0.0,0.0
1993-02-04 00:00:00-05:00,25.711973,25.783445,25.426086,25.729841,531500,0.0,0.0,0.0


In [115]:
print(market_df.index[0].date())

1993-01-29


In [116]:
months = []
days = []
years = []
index = market_df.index
new_index = []
for ts in index:
    new_index.append(ts.date())
for date in index:
    months.append(date.month)
    days.append(date.day)
    years.append(date.year)
months = pd.Series(months,index=index)
years = pd.Series(years,index=index)
days = pd.Series(days,index=index)
dates = pd.Series(new_index,index=index)
market_df['Month'] = months
market_df['Day'] = days
market_df['Year'] = years
market_df['Date'] = dates
market_df.reset_index(drop=True,inplace=True)

market_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Month,Day,Year,Date
0,25.140211,25.140211,25.015135,25.122343,1003200,0.0,0.0,0.0,1,29,1993,1993-01-29
1,25.140212,25.301023,25.140212,25.301023,480500,0.0,0.0,0.0,2,1,1993,1993-02-01
2,25.283158,25.372497,25.229554,25.354630,201300,0.0,0.0,0.0,2,2,1993,1993-02-02
3,25.390369,25.640520,25.372501,25.622652,529400,0.0,0.0,0.0,2,3,1993,1993-02-03
4,25.711973,25.783445,25.426086,25.729841,531500,0.0,0.0,0.0,2,4,1993,1993-02-04


In [117]:
market_df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Month,Day,Year,Date
7621,408.910004,413.720001,408.640015,412.630005,87844000,0.0,0.0,0.0,5,5,2023,2023-05-05
7622,412.970001,413.239990,411.279999,412.739990,50046800,0.0,0.0,0.0,5,8,2023,2023-05-08
7623,411.130005,412.089996,410.690002,410.929993,49220100,0.0,0.0,0.0,5,9,2023,2023-05-09
7624,413.880005,414.540009,408.869995,412.850006,96142900,0.0,0.0,0.0,5,10,2023,2023-05-10
7625,411.950012,412.429993,409.970001,412.170013,61947377,0.0,0.0,0.0,5,11,2023,2023-05-11


In [151]:
market_df.to_csv('market_data.csv',index=False)

# End

# Read in market data

In [119]:
market_df = pd.read_csv('market_data.csv')
market_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Month,Day,Year,Date
0,25.140211,25.140211,25.015135,25.122343,1003200,0.0,0.0,0.0,1,29,1993,1993-01-29
1,25.140212,25.301023,25.140212,25.301023,480500,0.0,0.0,0.0,2,1,1993,1993-02-01
2,25.283158,25.372497,25.229554,25.354630,201300,0.0,0.0,0.0,2,2,1993,1993-02-02
3,25.390369,25.640520,25.372501,25.622652,529400,0.0,0.0,0.0,2,3,1993,1993-02-03
4,25.711973,25.783445,25.426086,25.729841,531500,0.0,0.0,0.0,2,4,1993,1993-02-04


# End

# Create monthly returns json and save it

In [120]:
monthly_asset_gb = data.groupby(['Year','Month'])

In [121]:
monthly_market_gb = market_df.groupby(['Year','Month'])

In [123]:
monthly_asset_gb.get_group((2009,12))

,Adj Close: A,Adj Close: AA,Adj Close: AAP,Adj Close: AB,Adj Close: ABB,Adj Close: ABC,Adj Close: ABEV,Adj Close: ABG,Adj Close: ABT,Adj Close: ACM,...,Volume: X,Volume: XOM,Volume: XPO,Volume: YPF,Volume: YUM,Volume: ZBH,Month,Day,Year,Date
0,19.640217,35.237846,37.049255,9.560406,12.084818,21.716007,2.183232,11.78,19.265392,27.797508,...,8529900.0,19985700,69683.0,3600.0,2844734.0,607906.0,12,29,2009,2009-12-29
1,20.138828,35.831367,37.276497,9.712652,12.072253,21.666595,2.174173,11.81,19.364801,27.402514,...,8010200.0,19860600,18722.0,2600.0,1886196.0,594413.0,12,30,2009,2009-12-30
2,20.119402,35.435684,36.794739,9.723032,11.996881,21.468946,2.172915,11.53,19.169525,27.155643,...,3589500.0,18852300,36648.0,11300.0,1790217.0,579169.0,12,31,2009,2009-12-31


In [140]:
monthly_rets = {}

for yr in range(2010,2023):
    monthly_rets[yr] = {}
    for month in range(1,13):
        group_indexer = (yr,month)
        monthly_rets[yr][month] = {}
        asset_gb = monthly_asset_gb.get_group(group_indexer)
        market_gb = monthly_market_gb.get_group(group_indexer)
        
        market_first_day = market_gb.head(1)
        market_last_day = market_gb.tail(1)
        market_month_open = market_first_day['Open'].iloc[0]
        market_month_close = market_last_day['Close'].iloc[0]
        market_month_ret = (market_month_close-market_month_open)/market_month_open
        monthly_rets[yr][month]['Market'] = market_month_ret
        
        asset_first_day = asset_gb.head(1)
        asset_last_day = asset_gb.tail(1)
        
        for s in symbols:
            asset_month_open = asset_first_day['Open: ' + s].iloc[0]
            asset_month_close = asset_last_day['Adj Close: ' + s].iloc[0]
            asset_month_ret = (asset_month_close-asset_month_open)/asset_month_open
            monthly_rets[yr][month][s] = asset_month_ret

In [141]:
#save monthly rets as json; yr -> month -> symbol -> return

import json

with open('monthly_returns.json', 'w') as fp:
    json.dump(monthly_rets, fp, indent=3)

# End

# Read in monthly returns json

In [142]:
with open('monthly_returns.json', 'r') as fp:
    monthly_rets = json.load(fp)

# End

#  Create yearly returns json and save it

In [147]:
yearly_asset_gb = data.groupby(['Year'])
yearly_market_gb = market_df.groupby(['Year'])

In [148]:
yearly_rets = {}
for yr in range(2010,2023):
    yearly_rets[yr] = {}
    asset_gp = yearly_asset_gb.get_group(yr)
    market_gp = yearly_market_gb.get_group(yr)
    
    market_first_day = market_gp.head(1)
    market_last_day = market_gp.tail(1)
    market_open = market_first_day['Open'].iloc[0]
    market_close = market_last_day['Close'].iloc[0]
    market_ret = (market_close-market_open)/market_open
    yearly_rets[yr]['Market'] = market_ret

    asset_first_day = asset_gp.head(1)
    asset_last_day = asset_gp.tail(1)

    for s in symbols:
        asset_open = asset_first_day['Open: ' + s].iloc[0]
        asset_close = asset_last_day['Adj Close: ' + s].iloc[0]
        asset_ret = (asset_close-asset_open)/asset_open
        yearly_rets[yr][s] = asset_ret

with open('yearly_returns.json', 'w') as fp:
    json.dump(yearly_rets, fp, indent=3)

# End 

# Read in yearly returns json

In [150]:
with open('yearly_returns.json', 'r') as fp:
    yearly_rets = json.load(fp)

# End